In [1]:
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144
import math
from static_grader import grader

In [2]:
%%bash 
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -n

mkdir: cannot create directory ‘pw-data’: File exists
--2019-02-16 13:59:09--  http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.216.102.155
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.216.102.155|:80... connected.
HTTP request sent, awaiting response... 403 Forbidden
2019-02-16 13:59:09 ERROR 403: Forbidden.

wget: option requires an argument -- 'n'
wget: illegal option -- `-nh'
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [3]:
import gzip 
import simplejson as json

In [4]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)
with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)


In [5]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

In [6]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

## Question 1: summary_statistics
Our beneficiary data (`scripts`) contains quantitative data on the number
Calculate the sum, mean, standard deviation, and quartile statistics for e

In [6]:
def describe(key):
    total = sum(script[key] for script in scripts)
    avg = total / len(scripts)
    s = math.sqrt(sum((script[key]-avg)**2 for script in scripts)/ (len(scripts)-1))
    t = round(len(scripts)/4)
    q25 = sorted(script[key] for script in scripts)[t]
    med = sorted(script[key] for script in scripts)[2*t]
    q75 = sorted(script[key] for script in scripts)[3*t]
    
    return (total, avg, s, q25, med, q75)

In [7]:
describe('items')

(4410054, 11.522744731217633, 33.11220959820685, 1, 3, 8)

In [8]:
summary = [('items', describe('items')),('quantity', describe('quantity')),('nic', describe('nic')),
           ('act_cost',describe('nic')),('act_cost', describe('act_cost'))]

In [19]:
grader.score.pw_summary_statistics(summary)

No question found: pw_summary_statistics


## Question 2: most_common_item
Often we are not interested in how data is distributed in our entire data set, but we are also interested in particular groups -- for example, how many items of each drug (i.e. 'bnf_name') were prescribed? Calculate the total items prescribed for each 'bnf_name'. What is the most commonly prescribed 'bnf_name' in our dataset?

To calculate this, we first need to split the dataset into groups corresponding with different values of 'bnf_name'. Then we can sum the number of items dispensed within each group. Finally, we can find the largest sum.
 
We will use 'bnf_name' to construct our groups. you should have *5619* unique values for 'bnf_name'.

In [10]:
len(set(script['bnf_name'] for script in scripts))

5619

In [9]:
bnf_names = set(script['bnf_name'] for script in scripts)
assert(len(bnf_names)) == 5619

We want to construct \"groups\" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from 'scripts'). We will construct a dictionary called 'groups', using 'bnf_names' as the keys. we will represent a group with a `list`, since we can easily append new members to the group. To split out `scripts` into groups by `'bfn_name'`, we should iterate over `scripts`, appending prescription dictionaries as we encounter them.

In [10]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    groups[script['bnf_name']].append(script)

Now that we have constructed our groups we should sum up `'items'` in each group and finde the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[(Foobar, 2000)]`

In [10]:
groups['Foodlink_Complete Food Drink+Fibre (Van)'][0]['items']

2

In [13]:
sum_items_groups = []
for name in groups.keys():
    sum_items_groups.append([name,sum(groups[name][n]['items'] for n in range(len(groups[name])))])

In [14]:
max(sum_items_groups)

['palmdoc (Reagent)_Strips', 6]

In [231]:
def takeSecond(elem):
    return elem[1]

In [16]:
max_item = sorted(sum_items_groups, key =takeSecond, reverse= True)[0]

In [17]:
max_item = [('Omeprazole_Cap E/C 20mg', 113826)]

In [17]:
grader.score.pw__most_common_item(max_item)

No question found: pw__most_common_item


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name)` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [18]:
fields =('bnf_name', 'items')

In [19]:
groups_outer = {field: {} for field in fields}

In [20]:
for key in groups_outer.keys():
    groups_outer[key] = {script[key]: script for script in scripts}

In [21]:
len(groups_outer['bnf_name'])

5619

In [11]:
def group_by_field(data, fields):
    groups = {field: {} for field in fields}
    for key in groups.keys():
        groups[key] = {d[key]: [] for d in data}
    for d in data:
        groups[key][d[key]].append(d)    
    return groups 

In [28]:
groups = group_by_field(scripts, ('bnf_name',))['bnf_name']

In [29]:
groups['Co-Magaldrox_Susp 195mg/220mg/5ml S/F']

[[{'bnf_code': '0101010G0AAABAB',
   'items': 2,
   'practice': 'N81013',
   'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
   'nic': 5.98,
   'act_cost': 5.56,
   'quantity': 1000}],
 [{'bnf_code': '0101010G0AAABAB',
   'items': 1,
   'practice': 'N81029',
   'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
   'nic': 2.99,
   'act_cost': 2.78,
   'quantity': 500}],
 [{'bnf_code': '0101010G0AAABAB',
   'items': 2,
   'practice': 'N81088',
   'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
   'nic': 5.98,
   'act_cost': 5.56,
   'quantity': 1000}],
 [{'bnf_code': '0101010G0AAABAB',
   'items': 6,
   'practice': 'A81017',
   'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
   'nic': 20.93,
   'act_cost': 19.45,
   'quantity': 3500}],
 [{'bnf_code': '0101010G0AAABAB',
   'items': 2,
   'practice': 'A81034',
   'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
   'nic': 5.98,
   'act_cost': 5.56,
   'quantity': 1000}],
 [{'bnf_code': '0101010G0AAABAB',
   'items': 10,

## Question 3: postal_totals 
Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescibed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post 
code.

**NOTE:** Some practices have multiple postal codes associated with them. USe the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices'`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [21]:
for practice in practices:
    if practice['code'] == 'K82019':
        print(practice)

{'code': 'K82019', 'name': 'THE MANDEVILLE PRACTICE', 'addr_1': 'THE MANDEVILLE PRACTICE', 'addr_2': 'HANNON ROAD', 'borough': 'AYLESBURY', 'village': 'BUCKINGHAMSHIRE', 'post_code': 'HP21 8TR'}
{'code': 'K82019', 'name': 'THE MANDEVILLE PRACTICE', 'addr_1': 'FISHERMEAD MEDICAL CENTRE', 'addr_2': 'FISHERMEAD BOULEVARD', 'borough': 'MILTON KEYNES', 'village': 'BUCKINGHAMSHIRE', 'post_code': 'MK6 2LR'}


In [30]:
codes = group_by_field(practices,('code',))['code']

In [28]:
n = 0
for practice in practices:
    if practice['code'] == 'A81001':
        n += 1
print(n)

2


###  Dictionary version:
we won't use this one

In [75]:
practice_postal = {practice['code']: {'post_code': practice['post_code']} for practice in practices}
for practice in practices:
    if practice['code'] in practice_postal:
        if practice['post_code'] < practice_postal[practice['code']]['post_code']:
            practice_postal[practice['code']]['post_code'] = practice['post_code']
    else:
        practice_postal[practice['code']]['post_code'] = practice['post_code']

###  List version:
we will use this one

In [12]:
practice_postal = {practice['code']: practice['post_code'] for practice in practices}
for practice in practices:
    if practice['code'] in practice_postal:
        if practice['post_code'] < practice_postal[practice['code']]:
            practice_postal[practice['code']] = practice['post_code']
    else:
        practice_postal[practice['code']] = practice['post_code']

In [13]:
assert practice_postal['K82019'] == 'HP21 8TR'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

Now we can join `practice_postal` to `scripts`

In [14]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script['practice']]

In [15]:
joined[0]

{'bnf_code': '0101010G0AAABAB',
 'items': 2,
 'practice': 'N81013',
 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
 'nic': 5.98,
 'act_cost': 5.56,
 'quantity': 1000,
 'post_code': 'SK11 6JL'}

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

To do that we use the `group_by_field` function defined above

In [16]:
group_by_post = group_by_field(joined, ('post_code',))['post_code'] 

In [17]:
group_by_post['SK11 6JL'][0]['items']

2

In [19]:
# make it a function (for a dictionary of the type we're using now)
def agg_sum(dic, field):
    sum_data_by_field = []
    for key in dic.keys():
        sum_data_by_field.append((key,sum(dic[key][n][field] for n in range(len(dic[key])))))
    return sum_data_by_field

In [20]:
sum_ = agg_sum(group_by_post, 'items');

In [76]:
sum_items_by_post = []
for key in group_by_post.keys():
    sum_items_by_post.append([key,sum(group_by_post[key][n]['items'] for n in range(len(group_by_post[key])))])
    #print([key, group_by_post[key][0][0]['items']] )

In [21]:
postal_totals = sorted(agg_sum(group_by_post, 'items'))[:100]

In [23]:
grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region
Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.


Now, we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. USe the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [139]:
%who

agg_sum	 bnf_	 bnf_names	 describe	 dict_	 f	 get_unique_agg	 grader	 group_by_field	 
group_by_post	 groups	 gzip	 i	 items_name_post	 joined	 json	 key	 math	 
matplotlib	 pairs	 post	 post_name_items	 post_name_sum	 postal_totals	 practice	 practice_postal	 practices	 
presc	 script	 scripts	 sns	 sum_	 summary	 unique_names	 


In [135]:
joined[3]

{'bnf_code': '0101021B0AAAPAP',
 'items': 3,
 'practice': 'N81013',
 'bnf_name': 'Sod Alginate/Pot Bicarb_Tab Chble 500mg',
 'nic': 9.21,
 'act_cost': 8.55,
 'quantity': 180,
 'post_code': 'SK11 6JL'}

In [24]:
group_by_post = group_by_field(joined, ('post_code',))['post_code']

In [147]:
test_by_name = group_by_field(group_by_post['B18 7AL'],('bnf_name',))['bnf_name']

In [164]:
[test_by_name['Aciclovir_Tab 200mg'][i]['items'] for i in range(len(test_by_name['Aciclovir_Tab 200mg']))]

[1, 1, 2]

In [167]:
pairs =[]
for name in test_by_name:
    pairs.append((name,sum([test_by_name[name][i]['items'] for i in range(len(test_by_name[name]))])))
   ## print(sum([test_by_name[name][i]['items'] for i in range(len(test_by_name[name]))]))

In [172]:
triads = []
for post in group_by_post.keys():
    test_by_name = group_by_field(group_by_post[post],('bnf_name',))['bnf_name']
    for name in test_by_name:
        triads.append((post,name,sum([test_by_name[name][i]['items'] for i in range(len(test_by_name[name]))])))

In [174]:
len(triads)

141196

In [176]:
total_items_by_bnf = triads
assert len(total_items_by_bnf) == 141196

Let's use `total_by_item_post` to find the maximum item total for each postal code. To do this, we will want to regroup `total_by_item_post` by `'post_code'` only, not by `('post_code','bnf_name')`. First let's turn `total_by_item_post` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in `total_by_item_post` after grouping it by `'post_code`.

In [182]:
diads = []
for post in group_by_post.keys():
    diads.append((post,sum([group_by_post[post][i]['items'] for i in range(len(group_by_post[post]))])))

In [194]:
assert len(total_items_by_post) == 118

In [193]:
total_items_by_post = {diads[i][0]: diads[i][1] for i in range(len(diads))}

In [196]:
total_items_by_post['B11 4BW']

20673

In [178]:
group_by_post['B11 4BW'][0]

{'bnf_code': '0101010R0BCAAAB',
 'items': 7,
 'practice': 'M85078',
 'bnf_name': 'Infacol_Susp 40mg/ml S/F',
 'nic': 21.68,
 'act_cost': 20.15,
 'quantity': 400,
 'post_code': 'B11 4BW'}

Now, we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the tem total and `'bnf_name'`.

In [197]:
group_by_post['B11 4BW']

[{'bnf_code': '0101010R0BCAAAB',
  'items': 7,
  'practice': 'M85078',
  'bnf_name': 'Infacol_Susp 40mg/ml S/F',
  'nic': 21.68,
  'act_cost': 20.15,
  'quantity': 400,
  'post_code': 'B11 4BW'},
 {'bnf_code': '0101021B0AAALAL',
  'items': 4,
  'practice': 'M85078',
  'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
  'nic': 14.33,
  'act_cost': 13.61,
  'quantity': 1400,
  'post_code': 'B11 4BW'},
 {'bnf_code': '0101021B0BEADAJ',
  'items': 5,
  'practice': 'M85078',
  'bnf_name': 'Gaviscon Infant_Sach 2g (Dual Pack) S/F',
  'nic': 24.1,
  'act_cost': 22.37,
  'quantity': 75,
  'post_code': 'B11 4BW'},
 {'bnf_code': '0101021B0BEAIAL',
  'items': 62,
  'practice': 'M85078',
  'bnf_name': 'Gaviscon Advance_Liq (Aniseed) (Reckitt)',
  'nic': 360.13,
  'act_cost': 334.57,
  'quantity': 31650,
  'post_code': 'B11 4BW'},
 {'bnf_code': '0101021B0BEBEAL',
  'items': 4,
  'practice': 'M85078',
  'bnf_name': 'Gaviscon Advance_Liq (Peppermint) S/F',
  'nic': 21.18,
  'act_cost': 19.65,
  'quantity':

In [203]:
triads_prop = []
for i in range(len(triads)):
    triads_prop.append((triads[i][0],triads[i][1],triads[i][2]/total_items_by_post[triads[i][0]]))  

In [209]:
post_item = {post: [] for post in set([triads[i][0] for i in range(len(triads))]) }

In [221]:
for i in range(len(triads)):
    post_item[triads[i][0]].append((triads[i][1],triads[i][2]))

In [235]:
max_by_post = []
for post in post_item:
    max_by_post.append((post,*max(post_item[post], key = takeSecond)))

In [238]:
triads_prop = []
for i in range(len(max_by_post)):
    triads_prop.append((max_by_post[i][0],max_by_post[i][1],max_by_post[i][2]/total_items_by_post[max_by_post[i][0]]))

In [239]:
sorted(triads_prop)[:100]

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.03415082474725487),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.02926161780958897),
 ('B21 9RY', 'Metformin HCl_Tab 500mg', 0.03549462254750369),
 ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.024095900880968663),
 ('B70 7AW', 'Paracet_Tab 500mg', 0.0266896608360023),
 ('BB11 2DL', 'Omeprazole_Cap E/C 20mg', 0.02884503434625684),
 ('BB2 1AX', 'Omeprazole_Cap E/C 20mg', 0.03645501521908402),
 ('BB3 1PY', 'Omeprazole_Cap E/C 20mg', 0.03428477088454342),
 ('BB4 5SL', 'Omeprazole_Cap E/C 20mg', 0.040696883081529876),
 ('BB7 2JG', 'Omeprazole_Cap E/C 20mg', 0.029471795446899183),
 ('BB8 0JZ', 'Atorvastatin_Tab 20mg', 0.022563442442074293),
 ('BB9 7SR', 'Omeprazole_Cap E/C 20mg', 0.023833193804939305),
 ('BD3 8QH', 'Atorvastatin_Tab 40mg', 0.03422179914326511),
 ('BH18 8EE', 'Omeprazole_Cap E/C 20mg', 0.029000583563798747),
 ('BH23 3AF', 'Omeprazole_Cap E/C 20mg', 0.03733292364418497),
 ('BL1 8TU', 'Omeprazole_Cap E/C 20mg', 0

In [240]:
grader.score.pw__items_by_region(sorted(triads_prop)[:100])

Your score:  1.0
